In [21]:
import os
import librosa
import math
import json

In [12]:
SAMPLE_RATE = 22050 
#standard value for music processing
DURATION = 30
#duration of the song in dataset. With gtzan dataset is always 30 for each song


In [24]:
def save_mfcc(dataset_path,json_path,number_of_mfcc = 13, n_fft = 2048,hop_length = 512,num_segments = 5):
    """
        dataset_path   : The path of the dataset,
        json_path      : The location where all the data processed in this function will stored such as genres,labels,mfcc values
        number_of_mfcc : The segments in mfcc graph,
        n_fft          : The number of samples taken (in say the conversion of analog to digital stuff),
        hop_length     : The number of units the function should move in calculating the mfcc,
        n_segments     : Chopping up a track in multiple segments so that we have more input data (100 isn't enough)
    """
    
    data = {
        "genres" : [], #inputs 
        "mfcc"   : [],
        "labels" : [], #outputs
        
    }
    
    for i,(root,dirs,files) in enumerate(os.walk(dataset_path)):
        """
            i    : The current level os.walk is in.
            Root : Path to the folder we are currently in,
            dirs : Directories present in that root aka sub-folders
            files : files in that root
            
            os.walk does it in the top-down manner. 
        """
        
        #ensure that we are not at the root level
        if root not in dataset_path:
            
            #saving the genre in data
            genre = root.split("/")[-1]
            data['genres'].append(genre)
            print(f'Genre{genre}')
            
            for f in files:
                #f only gives the filename such as Never gonna give up.wav
                file_path = os.path.join(root,f)
                signal,sr = librosa.load(file_path,sr = SAMPLE_RATE)
                
                number_of_samples_per_track = SAMPLE_RATE * DURATION
                samples_per_segment = int(number_of_samples_per_track / num_segments)
                expected_number_of_mfcc_vectors_per_segment = math.ceil(number_of_samples_per_track/hop_length)
                
                #processing segments and extracting mfcc data
                for segment in range(num_segments):
                    start_sample = samples_per_segment * segment
                    finish_sample = start_sample + samples_per_segment
                    
                    mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                                sr = sr,
                                                n_fft = n_fft,
                                                n_mfcc = number_of_mfcc,
                                                hop_length = hop_length)
                    mfcc = mfcc.T
                    
                    if len(mfcc) == expected_number_of_mfcc_vectors_per_segment:
                        data['mfcc'].append(mfcc.tolist())
                        data['labels'].append(i - 1)
                        #why i - 1 because i = 0 is going to be the root folder which we don't want
                        print("{}, segment:{}".format(file_path, d+1))
    
    with open(json_path,'w') as fp:
        json.dump(data,fp,indent = 4)


In [25]:
save_mfcc('/home/ritvik/Final Year Project/Testing/genres',
          '/home/ritvik/Final Year Project/Testing/data.json')

Genrerock
Genrereggae
Genrecountry
Genrepop
Genredisco
Genrejazz


KeyboardInterrupt: 